In [1]:
import pandas as pd
import arcpy
import os
from arcpy.sa import *
from arcpy import env
import numpy as np
import math
from arcpy import env
from arcpy.sa import *

In [5]:
#find the database
arcpy.env.workspace =r"D:\Maxim\Heihe\Heihe.gdb"

In [6]:
rasters = arcpy.ListRasters(":.tif")
raster_list = arcpy.ListRasters("") 

In [7]:
print(raster_list)

['LandCover_Heide3', 'Permafrost_Heide33', 'Slope_Heide', 'SoilTexture_Heide3', 'solar_radia3', 'Vegetation_Heide33', 'DEM_Heide3', 'Solar_radiation_heihe_nw', 'Aspect_Heide']


Raster Normalisation

In [8]:
#call the raster 
Slope=Raster("Slope_Heide")
#calculating the normalisation
Slope_N= ((Slope - Slope.minimum)/(Slope.maximum - Slope.minimum))

In [9]:
#make the same calculation for all the other raster
Aspect=Raster("Aspect_Heide")
Aspect_N= ((Aspect - Aspect.minimum)/(Aspect.maximum - Aspect.minimum))

Solar_radiation=Raster("Solar_radiation_heihe_nw")
Solar_radiation_N= ((Solar_radiation - Solar_radiation.minimum)/(Solar_radiation.maximum -Solar_radiation.minimum))


LandCover=Raster("LandCover_Heide3")
LandCover_N = ((LandCover -LandCover.minimum)/(LandCover.maximum - LandCover.minimum))


SoilTexture =Raster("SoilTexture_Heide3")
SoilTexture_N = ((SoilTexture - SoilTexture.minimum)/(SoilTexture.maximum - SoilTexture.minimum))


Vegetation =Raster("Vegetation_Heide33")
Vegetation_N = ((Vegetation -Vegetation.minimum)/(Vegetation.maximum - Vegetation.minimum))


DEM=Raster("DEM_Heide3")
DEM_N = ((DEM - DEM.minimum)/(DEM.maximum - DEM.minimum))


In [10]:
#find the path of the Excel file containing the probaabilitys
file =('D:\Maxim\masterthesis\Probability_Heide3.xlsx')

In [11]:
Data = pd.read_excel(file)
Data

,Case,Slope,Aspect,Solar_radaiation,DEM,Landcover,Soiltexture,Vegetation,Total
0,Case 1,0.2500,0.0625,0.1250,0.1250,0.0625,0.2500,0.1250,1
1,Case2,0.0625,0.1250,0.1250,0.0625,0.2500,0.1250,0.2500,1
2,Case3,0.1250,0.1250,0.0625,0.2500,0.1250,0.2500,0.0625,1
3,Case4,0.1250,0.0625,0.2500,0.1250,0.2500,0.0625,0.1250,1
4,Case5,0.0625,0.2500,0.1250,0.2500,0.0625,0.1250,0.1250,1
5,Case6,0.2500,0.1250,0.2500,0.0625,0.1250,0.1250,0.0625,1
6,Case7,0.1250,0.2500,0.0625,0.1250,0.1250,0.0625,0.2500,1
7,Case8,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1
8,Case9,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1
9,Case10,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1


In [14]:
#calculating of the different layer thinkness using the table of weight
ALT_list=[]
for i,rows in Data.iterrows():
    ALT=(rows["Slope"]*Slope_N + rows["Aspect"]*Aspect_N + rows["Solar_radaiation"]*Solar_radiation_N + rows["DEM"]*DEM_N + rows["Landcover"]* LandCover_N + rows["Soiltexture"]*SoilTexture_N  + rows["Vegetation"]*Vegetation_N)
    #add in the liste
    ALT_list.append(ALT)
    ALT
    #save als tif file
    ALT.save(r"D:\Maxim\masterthesis\YAK_TEST\ALTheide_new"+str(i)+".tif")

In [15]:
#Normalisation of the 14 result
ALT_list_N=[]
liste=[0,1,2,3,5,4,6,7,8,9,10,11,12,13]
for i in  liste:
    ALT_N = ( ALT_list[i]) - ( ALT_list[i].minimum)/ (( ALT_list[i].maximum) - ( ALT_list[i].minimum)) 
    #add in the  liste
    ALT_N
    ALT_list_N.append( ALT_N)
    #save als tif file
    ALT_N.save(r"D:\Maxim\masterthesis\YAK_TEST\ALTheide_new_N" + str(i)+".tif") 

In [16]:
#Original file 
ALT_original=Raster("Permafrost_Heide33")

In [18]:
#Normalisation of the Original file
ALT_original_N= Raster((ALT_original - ALT_original.minimum)/(ALT_original.maximum - ALT_original.minimum))
#save als tif file
ALT_original_N.save(r"D:\Maxim\masterthesis\YAK_TEST\YAK_TEST\ALT_Originalheide_new_N." + str(i)+".tif") 

ROOT MEAN SQUARE ERROR

In [19]:
#Root mean square error 
RMSE =[]
for i in  liste:
    A = arcpy.sa.Arithmetic ( ALT_list[i],ALT_original,"Minus", "FirstOf", "FirstOf")
    X = arcpy.sa.Arithmetic(A,A, "Multiply", "FirstOf", "FirstOf")
    rmse = np.square(X.mean)
    RMSE.append(rmse) 

In [20]:
#Root mean square of the rasters after the normalisation
RMSE_N=[]
for i in  liste:
    A = arcpy.sa.Arithmetic (ALT_list_N[i], ALT_original_N,"Minus", "FirstOf", "FirstOf")
    X = arcpy.sa.Arithmetic(A,A, "Multiply", "FirstOf", "FirstOf")
    rmse = np.square(X.mean)
    RMSE_N.append(rmse) 

Creating a table

In [21]:
#Rows and colums
case=["case1","case2","case3","case4","case5","case6","case7","case8","case9","case10","case11","case12","case13","case14"]
columns=["RMSE","RMSE_N"]

In [22]:

#putting the result in a dataframe
d1 = {"case": case, "RMSE":RMSE, "RMSE_N" : RMSE_N }

In [23]:
#panda dataframe
df= pd.DataFrame(d1)
df.to_excel("D:\Maxim\masterthesis\YAK_TEST\Dataframe_case.xlsx") 

In [24]:
df

,case,RMSE,RMSE_N
0,case1,0.005822,0.010261
1,case2,0.006673,0.016492
2,case3,0.004880,0.006553
3,case4,0.005673,0.014817
4,case5,0.006377,0.008215
5,case6,0.006037,0.009963
6,case7,0.007422,0.012429
7,case8,0.031873,0.031906
8,case9,0.031516,0.032284
9,case10,0.016749,0.017740


Best case

In [25]:
df["RMSE_N"].idxmin()

10